In [1]:
import pandas as pd 
from absbox import API,save
from absbox.local.china import SPV,show,plot_bond,plot_bonds
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:,}'.format
#localAPI = API("https://absbox.org/api/latest")
localAPI = API("http://localhost:8081")

ERROR:root:中文字体载入失败
/tmp/ipykernel_1085505/3432128074.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
from absbox.local.china import SPV


asset1 = ["按揭贷款"
        ,{"放款金额":120,"放款利率":["固定",0.045],"初始期限":30
          ,"频率":"每月","类型":"等额本金","放款日":"2021-02-01"}
          ,{"当前余额":120
          ,"当前利率":0.08
          ,"剩余期限":20
          ,"状态":"正常"}]
asset2 = ["按揭贷款"
        ,{"放款金额":120,"放款利率":["固定",0.045],"初始期限":30
          ,"频率":"每月","类型":"等额本金","放款日":"2021-02-01"}
          ,{"当前余额":120
          ,"当前利率":0.08
          ,"剩余期限":20
          ,"状态":"正常"}]

test01 = SPV(
    "Apply Assumption on Loan Level"
    ,{"封包日":"2021-03-31","起息日":"2021-06-15","首次兑付日":"2021-07-26"
      ,"法定到期日":"2060-12-01","收款频率":"月末","付款频率":["每月",26]}
    ,{'清单':[asset1, asset2]}
    ,(("账户01",{"余额":0}),)
    ,(("A1",{"当前余额":100
             ,"当前利率":0.07
             ,"初始余额":100
             ,"初始利率":0.07
             ,"起息日":"2020-01-03"
             ,"利率":{"固定":0.08}
             ,"债券类型":{"过手摊还":None}})
      ,("B",{"当前余额":20
             ,"当前利率":0.0
             ,"初始余额":100
             ,"初始利率":0.07
             ,"起息日":"2020-01-03"
             ,"利率":{"固定":0.00}
             ,"债券类型":{"权益":None}
             }))
    ,(("信托费用",{"类型":{"固定费用":30}}),)
    ,{"未违约":[
         ["支付费用",["账户01"],['信托费用']]
         ,["支付利息","账户01",["A1"]]
         ,["支付本金","账户01",["A1"]]
         ,["支付本金","账户01",["B"]]
         ,["支付收益","账户01","B"]
     ]}
    ,(["利息回款","账户01"]
      ,["本金回款","账户01"]
      ,["早偿回款","账户01"]
      ,["回收回款","账户01"])
    ,None
    ,None
)

Asset01Assump = [{"CPR":0.1}
                ,{"回收":(0.5,1)}
                ,{"CDR":0.0}]

Asset02Assump = [{"CPR":0.1}
                ,{"回收":(0.5,1)}
                ,{"CDR":0.01}]


myAssumption = ([([0],Asset01Assump),([1],Asset02Assump)],
                [{"利率":["LPR5Y",[["2022-01-01",0.05]]]}])


r = localAPI.run(test01
             ,assumptions= {"00":Asset01Assump,"01":Asset02Assump}           
             #,assumptions= []
             ,read=True)


r['00']

{'bonds': OrderedDict([('A1',
                             余额   利息    本金  执行利率  本息合计  \
               日期                                        
               2021-07-26 87.0  0.78 13.0  0.07  13.78   
               2021-08-26 73.51 0.51 13.49 0.07  14.0    
               2021-09-26 60.14 0.43 13.37 0.07  13.8    
               2021-10-26 46.94 0.34 13.2  0.07  13.54   
               2021-11-26 33.99 0.27 12.95 0.07  13.22   
               2021-12-26 21.16 0.19 12.83 0.07  13.02   
               2022-01-26 8.58  0.12 12.58 0.07  12.7    
               2022-02-26 0.0   0.05 8.58  0.07  8.63    
               
                                                                                 备注  
               日期                                                                    
               2021-07-26  [<PayInt:["A1"],Just 0.00>, <PayPrin:["A1"],Just 87.00>]  
               2021-08-26  [<PayInt:["A1"],Just 0.00>, <PayPrin:["A1"],Just 73.51>]  
               2021-09-26  [<P

In [3]:
import json
json.loads(localAPI.build_req(test01
             ,assumptions= {"00":Asset01Assump,"01":Asset02Assump}           
             #,assumptions= []
             ,read=True))['assump']

{'tag': 'Multiple',
 'contents': {'00': {'tag': 'PoolLevel',
   'contents': [{'tag': 'PrepaymentCPR', 'contents': 0.1},
    {'tag': 'Recovery', 'contents': [0.5, 1]},
    {'tag': 'DefaultCDR', 'contents': 0.0}]},
  '01': {'tag': 'PoolLevel',
   'contents': [{'tag': 'PrepaymentCPR', 'contents': 0.1},
    {'tag': 'Recovery', 'contents': [0.5, 1]},
    {'tag': 'DefaultCDR', 'contents': 0.01}]}}}